# Dummy 생성

In [ ]:
import json
import os
import sys
import numpy as np
import librosa
import librosa.display
import matplotlib
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
import json
import os
import numpy as np

def extract_features_dummy(file_path, result_folder):
    """
    Extract features and save plots to respective folders.
    """
    # 결과 폴더 생성
    os.makedirs(result_folder, exist_ok=True)
    
    # 파일 이름 추출
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 각 특징별 폴더 생성 및 이미지 경로 설정
    def get_image_path(feature_name):
        feature_folder = os.path.join(result_folder, feature_name)
        os.makedirs(feature_folder, exist_ok=True)
        return os.path.join(feature_folder, f"{file_name}.png")

    # 예시 특징 데이터
    return {
        "file_path": file_path,
        "features": {
            "zero_crossing_rate": 0.012,
            "harmonic_to_noise_ratio": 0.75,
            "spectral_flatness": 0.3,
            "spectral_rolloff": 2800.0,
            "rms_energy": 0.05,
            "entropy": 1.45,
            "waveform": get_image_path("waveform"),
            "envelope": get_image_path("envelope"),
            "cepstrum": get_image_path("cepstrum"),
            "mel_spectrogram_with_axes": get_image_path("mel_spectrogram_with_axes"),
            "mel_spectrogram_no_axes": get_image_path("mel_spectrogram_no_axes"),
            "mfcc": get_image_path("mfcc"),
            "linear_spectrogram_with_axes": get_image_path("linear_spectrogram_with_axes"),
            "linear_spectrogram_no_axes": get_image_path("linear_spectrogram_no_axes"),
            "chroma_features": get_image_path("chroma_features"),
            "spectral_centroid": get_image_path("spectral_centroid"),
            "spectral_bandwidth": get_image_path("spectral_bandwidth"),
            "power_spectrum": get_image_path("power_spectrum"),
            "std": 0.123,
            "avg": 0.456
        },
    }

def create_json_structure(data_entries, output_json_path):
    """
    Create a JSON file with structured data entries.
    """
    data = {"audio_features": data_entries}
    with open(output_json_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def test_main():
    """
    Main function to process all files and extract features.
    """
    classes = ["ToyCar", "ToyTrain", "bearing", "fan", "gearbox", "slider", "valve"]
    data_type = ['test', 'train']
    all_entries = []

    for class_name in classes:
        for data in data_type:
            folder_path = f'../unziped/dev/{class_name}/{data}'
            result_folder = os.path.join(folder_path, "Feature_Extraction_Results")
            os.makedirs(result_folder, exist_ok=True)
            
            # 모든 wav 파일 처리
            wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
            
            for wav_file in wav_files:
                file_path = os.path.join(folder_path, wav_file)
                features = extract_features_dummy(file_path, result_folder)
                
                # 섹션 정보 추출 (파일명에서 'section_' 뒤의 두 글자)
                if 'section_' in wav_file:
                    section_start = wav_file.find('section_') + len('section_')
                    section = wav_file[section_start:section_start+2]
                else:
                    section = '00'  # 기본값

                # note 추가 (자동 생성)
                note = f"This is a {class_name} machine, data type: {data}, file: {wav_file}"
                
                entry = {
                    "file_path": features["file_path"],
                    "machine_type": class_name,
                    "section": section,
                    "domain": data,  # 'train' 또는 'test'
                    "features": features["features"],
                    "note": note  # 주석 추가
                }
                all_entries.append(entry)
    
    # JSON 파일로 저장
    output_json_path = 'audio_features_dummy.json'
    create_json_structure(all_entries, output_json_path)
    print(f"JSON 파일이 '{output_json_path}' 경로에 저장되었습니다.")



# Image 경로와 특성들만 계산함

In [ ]:
import json
import os
import sys
from tqdm import tqdm

def load_existing_json(json_path):
    """
    기존 JSON 파일을 로드합니다.

    Parameters:
        json_path (str): JSON 파일 경로.

    Returns:
        dict: JSON 데이터.
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"JSON 파일 로드 중 오류 발생: {e}", file=sys.stderr)
        sys.exit(1)

def save_updated_json(data, json_path):
    """
    업데이트된 JSON 데이터를 저장합니다.

    Parameters:
        data (dict): 업데이트된 JSON 데이터.
        json_path (str): 저장할 JSON 파일 경로.
    """
    try:
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)  # 한글 지원을 위해 ensure_ascii=False 추가
        print(f"업데이트된 JSON 파일이 '{json_path}' 경로에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}", file=sys.stderr)
        sys.exit(1)

def construct_image_paths(file_path):
    """
    오디오 파일 경로를 기반으로 이미지 경로를 생성합니다.

    Parameters:
        file_path (str): 오디오 파일의 경로.

    Returns:
        dict: 각 특징별 이미지 경로.
    """
    features = [
        "waveform",
        "envelope",
        "cepstrum",
        "mel_spectrogram_with_axes",
        "mel_spectrogram_no_axes",
        "mfcc",
        "linear_spectrogram_with_axes",
        "linear_spectrogram_no_axes",
        "chroma_features",
        "spectral_centroid",
        "spectral_bandwidth",
        "power_spectrum"
    ]

    feature_paths = {}
    # Feature_Extraction_Results 폴더 경로
    result_folder = os.path.join(os.path.dirname(file_path), "Feature_Extraction_Results")

    # 파일 이름 추출 (확장자 제거)
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    for feature in features:
        image_path = os.path.join(result_folder, feature, f"{file_name}.png")
        # 상대 경로로 변환 (JSON에 저장할 경로)
        relative_image_path = os.path.relpath(image_path)
        feature_paths[feature] = relative_image_path.replace("\\", "/")  # 윈도우 호환성 위해 슬래시 변환

    return feature_paths

def create_reimaged_json(original_json_path, new_json_path):
    """
    기존 JSON 파일을 기반으로 새로운 JSON 파일을 생성합니다.
    이미지는 생성하지 않고 경로만 포함하며, "note" 필드는 기존 값을 유지합니다.

    Parameters:
        original_json_path (str): 기존 JSON 파일 경로.
        new_json_path (str): 새로 저장할 JSON 파일 경로.
    """
    data = load_existing_json(original_json_path)
    audio_features = data.get("audio_features", [])

    if not audio_features:
        print(f"'{original_json_path}'에 'audio_features' 데이터가 없습니다.", file=sys.stderr)
        sys.exit(1)

    updated_audio_features = []

    for entry in tqdm(audio_features, desc="Processing entries"):
        file_path = entry.get("file_path")
        if not file_path:
            print(f"파일 경로가 없는 엔트리 발견: {entry}", file=sys.stderr)
            continue

        # 이미지 경로만 재구성
        new_features = {}
        # 기존의 수치적 특징 유지
        numerical_features = {k: v for k, v in entry.get("features", {}).items()
                              if k not in [
                                  "waveform",
                                  "envelope",
                                  "cepstrum",
                                  "mel_spectrogram_with_axes",
                                  "mel_spectrogram_no_axes",
                                  "mfcc",
                                  "linear_spectrogram_with_axes",
                                  "linear_spectrogram_no_axes",
                                  "chroma_features",
                                  "spectral_centroid",
                                  "spectral_bandwidth",
                                  "power_spectrum"
                              ]}
        new_features.update(numerical_features)

        # 이미지 경로 재구성
        image_features = construct_image_paths(file_path)
        new_features.update(image_features)

        # "note" 필드 유지: 기존에 있으면 유지, 없으면 빈 문자열
        note = entry.get("note", "")

        # 새로운 엔트리 구성
        updated_entry = {
            "file_path": file_path,
            "machine_type": entry.get("machine_type", ""),
            "section": entry.get("section", ""),
            "domain": entry.get("domain", ""),
            "features": new_features,
            "note": note
        }

        updated_audio_features.append(updated_entry)

    # 새로운 JSON 구조 생성
    new_data = {"audio_features": updated_audio_features}

    # 새로운 JSON 파일 저장
    save_updated_json(new_data, new_json_path)

def main():
    original_json_path = 'audio_features_backup.json'
    new_json_path = 'audio_features.json'

    if not os.path.exists(original_json_path):
        print(f"`audio_features_backup.json` 파일을 찾을 수 없습니다: {original_json_path}", file=sys.stderr)
        sys.exit(1)

    create_reimaged_json(original_json_path, new_json_path)

if __name__ == "__main__":
    main()


# Image와 그에 대항하는 parameter만 재추출.

```json
{
    "audio_features": [
        {
            "file_path": "../unziped/dev/ToyCar/test/section_00_target_test_normal_0037_car_E2_spd_28V_mic_2.wav",
            "machine_type": "ToyCar",
            "section": "00",
            "domain": "test",
            "features": {
                "zero_crossing_rate": 0.0493423786569149,
                "harmonic_to_noise_ratio": 2.493234395980835,
                "spectral_flatness": 0.005233748350292444,
                "spectral_rolloff": 1905.6474401595744,
                "rms_energy": 0.02990570478141308,
                "entropy": -2320.7347731695854,
                "std": 0.030560411512851715,
                "avg": -1.4019012724020286e-06,
                "waveform": {
                    "path": "../unziped/dev/ToyCar/test/Feature_Extraction_Results/waveform/section_00_target_test_normal_0037_car_E2_spd_28V_mic_2.png",
                    "librosa_parameters": {},
                    "plot_parameters": {
                        "figsize": [8, 4],
                        "dpi": 100,
                        "file_format": "png"
                    }
                },
                "envelope": {
                    "path": "../unziped/dev/ToyCar/test/Feature_Extraction_Results/envelope/section_00_target_test_normal_0037_car_E2_spd_28V_mic_2.png",
                    "librosa_parameters": {},
                    "plot_parameters": {
                        "figsize": [8, 4],
                        "dpi": 100,
                        "file_format": "png"
                    }
                },
                ...
            },
            "note": ""
        },
        ...
    ]
}

```

In [ ]:
import json
import os
import sys
import numpy as np
import librosa
import librosa.display
import matplotlib
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
import shutil
from tqdm import tqdm  # 진행 상황 표시를 위한 tqdm 추가

# Matplotlib 백엔드를 'Agg'로 설정 (비대화형 백엔드)
matplotlib.use('Agg')

def extract_image_features(y, sr, file_name, result_folder, min_freq=0, max_freq=8000, normalization='min-max'):
    """
    Extract only image features from an audio signal and save corresponding plots along with their parameters.

    Parameters:
        y (np.ndarray): Audio time series.
        sr (int): Sampling rate.
        file_name (str): Base name of the audio file.
        result_folder (str): Directory to save feature plots.
        min_freq (float): Minimum frequency for plotting.
        max_freq (float): Maximum frequency for plotting.
        normalization (str): Type of normalization for MFCC.

    Returns:
        dict: Dictionary containing updated image feature paths and their parameters.
    """
    features = {}
    
    try:
        # 결과 폴더 생성
        os.makedirs(result_folder, exist_ok=True)

        # 각 특징별로 폴더 생성 및 이미지 저장 함수
        def save_plot(fig, feature_name, librosa_params, plot_params):
            feature_folder = os.path.join(result_folder, feature_name)
            os.makedirs(feature_folder, exist_ok=True)
            image_path = os.path.join(feature_folder, f"{file_name}.png")
            fig.savefig(image_path, bbox_inches='tight', pad_inches=0)
            plt.close(fig)
            # 상대 경로로 변환 (JSON에 저장할 경로)
            relative_image_path = os.path.relpath(image_path).replace("\\", "/")
            return {
                "path": relative_image_path,
                "librosa_parameters": librosa_params,
                "plot_parameters": plot_params
            }

        # 8. Waveform Plot
        librosa_params = {}  # Waveform은 추가 파라미터 없음
        plot_params = {
            "figsize": [8, 4],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_waveform(y, sr)
        waveform_info = save_plot(fig, 'waveform', librosa_params, plot_params)
        features['waveform'] = waveform_info

        # 9. Envelope Plot
        librosa_params = {}  # Envelope는 추가 파라미터 없음
        plot_params = {
            "figsize": [8, 4],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_envelope(y, sr)
        envelope_info = save_plot(fig, 'envelope', librosa_params, plot_params)
        features['envelope'] = envelope_info

        # 10. Cepstrum Plot
        librosa_params = {
            "min_freq": min_freq,
            "max_freq": max_freq
        }
        plot_params = {
            "figsize": [8, 4],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_cepstrum(y, sr, min_freq=min_freq, max_freq=max_freq)
        cepstrum_info = save_plot(fig, 'cepstrum', librosa_params, plot_params)
        features['cepstrum'] = cepstrum_info

        # 11. Mel Spectrogram Plot
        # (a) With axes
        librosa_params = {
            "fmin": 0,
            "fmax": 8000
        }
        plot_params = {
            "figsize": [6.4, 5.12],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_mel_spectrogram(y, sr, display_axes=True)
        mel_spectrogram_with_axes_info = save_plot(fig, 'mel_spectrogram_with_axes', librosa_params, plot_params)
        features['mel_spectrogram_with_axes'] = mel_spectrogram_with_axes_info

        # (b) Without axes
        librosa_params = {
            "fmin": 0,
            "fmax": 8000
        }
        plot_params = {
            "figsize": [5.12, 5.12],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_mel_spectrogram(y, sr, display_axes=False)
        mel_spectrogram_no_axes_info = save_plot(fig, 'mel_spectrogram_no_axes', librosa_params, plot_params)
        features['mel_spectrogram_no_axes'] = mel_spectrogram_no_axes_info

        # 12. MFCC Plot
        librosa_params = {
            "n_mfcc": 20,
            "normalization": normalization
        }
        plot_params = {
            "figsize": [8, 4],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_mfcc(y, sr, normalization=normalization)
        mfcc_info = save_plot(fig, 'mfcc', librosa_params, plot_params)
        features['mfcc'] = mfcc_info

        # 13. Linear Spectrogram Plot
        # (a) With axes
        librosa_params = {
            "n_fft": 256,
            "hop_length": 256,
            "window": "hann"
        }
        plot_params = {
            "figsize": [6.4, 5.12],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_linear_spectrogram(y, sr, n_fft=256, hop_length=256, display_axes=True)
        linear_spectrogram_with_axes_info = save_plot(fig, 'linear_spectrogram_with_axes', librosa_params, plot_params)
        features['linear_spectrogram_with_axes'] = linear_spectrogram_with_axes_info

        # (b) Without axes
        librosa_params = {
            "n_fft": 256,
            "hop_length": 256,
            "window": "hann"
        }
        plot_params = {
            "figsize": [5.12, 5.12],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_linear_spectrogram(y, sr, n_fft=256, hop_length=256, display_axes=False)
        linear_spectrogram_no_axes_info = save_plot(fig, 'linear_spectrogram_no_axes', librosa_params, plot_params)
        features['linear_spectrogram_no_axes'] = linear_spectrogram_no_axes_info

        # 14. Chroma Features Plot
        librosa_params = {
            "hop_length": 512
        }
        plot_params = {
            "figsize": [8, 4],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_chroma_features(y, sr)
        chroma_features_info = save_plot(fig, 'chroma_features', librosa_params, plot_params)
        features['chroma_features'] = chroma_features_info

        # 15. Spectral Centroid Plot
        librosa_params = {
            "hop_length": 512
        }
        plot_params = {
            "figsize": [8, 4],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_spectral_centroid(y, sr)
        spectral_centroid_info = save_plot(fig, 'spectral_centroid', librosa_params, plot_params)
        features['spectral_centroid'] = spectral_centroid_info

        # 16. Spectral Bandwidth Plot
        librosa_params = {
            "hop_length": 512
        }
        plot_params = {
            "figsize": [8, 4],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_spectral_bandwidth(y, sr)
        spectral_bandwidth_info = save_plot(fig, 'spectral_bandwidth', librosa_params, plot_params)
        features['spectral_bandwidth'] = spectral_bandwidth_info

        # 17. Power Spectrum Plot
        librosa_params = {
            "n_fft": len(y),
            "hop_length": None
        }
        plot_params = {
            "figsize": [8, 4],
            "dpi": 100,
            "file_format": "png"
        }
        fig, ax = compute_and_plot_power_spectrum(y, sr)
        power_spectrum_info = save_plot(fig, 'power_spectrum', librosa_params, plot_params)
        features['power_spectrum'] = power_spectrum_info

    except Exception as e:
        print(f"이미지 특징 추출 중 오류 발생: {file_name}\n오류 내용: {e}", file=sys.stderr)
        return None

    return features

def backup_json(original_path, backup_path):
    """
    백업할 JSON 파일을 복사합니다.

    Parameters:
        original_path (str): 원본 JSON 파일 경로.
        backup_path (str): 백업할 JSON 파일 경로.
    """
    try:
        shutil.copyfile(original_path, backup_path)
        print(f"백업 완료: '{backup_path}'")
    except Exception as e:
        print(f"백업 중 오류 발생: {e}", file=sys.stderr)
        sys.exit(1)

def load_existing_json(json_path):
    """
    기존 JSON 파일을 로드합니다.

    Parameters:
        json_path (str): JSON 파일 경로.

    Returns:
        dict: JSON 데이터.
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"JSON 파일 로드 중 오류 발생: {e}", file=sys.stderr)
        sys.exit(1)

def save_updated_json(data, json_path):
    """
    업데이트된 JSON 데이터를 저장합니다.

    Parameters:
        data (dict): 업데이트된 JSON 데이터.
        json_path (str): 저장할 JSON 파일 경로.
    """
    try:
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)  # 한글 지원을 위해 ensure_ascii=False 추가
        print(f"업데이트된 JSON 파일이 '{json_path}' 경로에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}", file=sys.stderr)
        sys.exit(1)

def process_entry(task):
    """
    개별 JSON 엔트리를 처리하여 이미지 특징을 재추출합니다.

    Parameters:
        task (tuple): (entry, base_dir)

    Returns:
        dict or None: 업데이트된 엔트리 또는 None
    """
    entry, base_dir = task
    file_path = entry.get("file_path")
    if not file_path:
        print(f"파일 경로가 없는 엔트리 발견: {entry}", file=sys.stderr)
        return None

    try:
        y, sr = librosa.load(file_path, sr=None)
    except Exception as e:
        print(f"오디오 파일을 로드하는 중 오류 발생: {file_path}\n오류 내용: {e}", file=sys.stderr)
        return None

    # 결과 폴더 경로
    result_folder = os.path.join(os.path.dirname(file_path), "Feature_Extraction_Results")

    # 파일 이름 추출
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 이미지 특징 재추출
    image_features = extract_image_features(
        y=y,
        sr=sr,
        file_name=file_name,
        result_folder=result_folder,
        min_freq=0,  # min_freq=0으로 설정
        max_freq=8000,
        normalization='min-max'
    )

    if image_features is None:
        return None

    # 기존 features 유지 (숫자 데이터)
    existing_features = entry.get("features", {})
    numerical_features = {k: v for k, v in existing_features.items() if not isinstance(v, str)}
    
    # 업데이트된 이미지 특징 반영
    updated_features = numerical_features
    updated_features.update(image_features)

    # note 필드 유지
    note = entry.get("note", "")

    # 업데이트된 엔트리 반환
    updated_entry = {
        "file_path": entry.get("file_path"),
        "machine_type": entry.get("machine_type", ""),
        "section": entry.get("section", ""),
        "domain": entry.get("domain", ""),
        "features": updated_features,
        "note": note
    }

    # 처리 로그 기록
    with open('processing_log.txt', 'a', encoding='utf-8') as log_file:
        log_file.write(f"Processed: {file_path}\n")

    return updated_entry

def create_json_structure(data_entries, output_json_path):
    """
    Create a JSON file with structured data entries.

    Parameters:
        data_entries (list): List of feature dictionaries.
        output_json_path (str): Path to save the JSON file.
    """
    def convert_numpy_types(obj):
        if isinstance(obj, np.float32) or isinstance(obj, np.float64):
            return float(obj)
        elif isinstance(obj, dict):
            return {k: convert_numpy_types(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(i) for i in obj]
        else:
            return obj

    # JSON 저장 전에 변환
    data = convert_numpy_types({"audio_features": data_entries})
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

def main():
    """
    메인 함수: 기존 JSON 파일을 백업하고, 이미지 특징을 재추출하여 업데이트합니다.
    """
    input_json_path = 'audio_features.json'  
    output_json_path = 'audio_features_re.json'  # 새로운 JSON 파일 이름
    backup_path = 'audio_features_backup.json'  # 작업 전에 기존 백업을 또 다른 백업으로 복사

    # 1. 추가 백업 생성 (안전성을 위해)
    if os.path.exists(input_json_path):
        backup_json(input_json_path, backup_path)
    else:
        print(f"audio_features JSON 파일을 찾을 수 없습니다: {input_json_path}", file=sys.stderr)
        sys.exit(1)

    # 2. backup JSON 로드
    data = load_existing_json(backup_path)
    audio_features = data.get("audio_features", [])

    if not audio_features:
        print("'audio_features_backup.json' 데이터가 없습니다.", file=sys.stderr)
        sys.exit(1)

    # 3. 멀티프로세싱을 위한 작업 목록 생성
    base_dir = '../unziped/dev'  # 필요 시 조정
    tasks = [(entry, base_dir) for entry in audio_features]

    # 4. 멀티프로세싱 풀 생성
    total_tasks = len(tasks)
    num_workers = max(1, cpu_count() - 1)  # 하나의 코어는 양보
    print(f"사용 가능한 CPU 코어 수: {cpu_count()}, 사용되는 코어 수: {num_workers}")
    print(f"총 처리할 엔트리 수: {total_tasks}")

    updated_entries = []

    # tqdm을 사용하여 진행 상황 표시
    with Pool(processes=num_workers) as pool:
        # apply_async와 tqdm을 조합하여 사용
        results = [pool.apply_async(process_entry, args=(task,)) for task in tasks]
        for r in tqdm(results, total=total_tasks, desc="Processing entries"):
            try:
                result = r.get()
                if result is not None:
                    updated_entries.append(result)
            except ZeroDivisionError as zde:
                print(f"ZeroDivisionError 처리 중: {zde}", file=sys.stderr)
            except Exception as e:
                print(f"예상치 못한 오류 발생: {e}", file=sys.stderr)

    # 줄바꿈 후 JSON 파일 저장 완료 메시지 출력
    print()  # 줄바꿈
    try:
        create_json_structure(updated_entries, output_json_path)
        print(f"JSON 파일이 '{output_json_path}' 경로에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}", file=sys.stderr)

# 플롯 함수들 (축 세부 조정)
def compute_and_plot_waveform(y: np.ndarray, sr: int):
    fig, ax = plt.subplots(figsize=(8, 4))  
    librosa.display.waveshow(y, sr=sr, ax=ax)
    ax.set_title('Waveform')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.grid(True)  # 격자 추가
    # 축 범위 설정
    ax.set_xlim(0, len(y)/sr)
    ax.set_ylim(np.min(y), np.max(y))
    plt.tight_layout()
    return fig, ax

def compute_and_plot_envelope(y: np.ndarray, sr: int):
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    times = librosa.times_like(onset_env, sr=sr)
    fig, ax = plt.subplots(figsize=(8, 4))  
    ax.plot(times, onset_env)
    ax.set_title('Envelope')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.grid(True)
    ax.set_xlim(0, len(y)/sr)
    ax.set_ylim(np.min(onset_env), np.max(onset_env))
    plt.tight_layout()
    return fig, ax

def compute_and_plot_cepstrum(y: np.ndarray, sr: int, min_freq: float, max_freq: float):
    spectrum = np.fft.fft(y)
    log_spectrum = np.log(np.abs(spectrum) + 1e-10)
    cepstrum = np.fft.ifft(log_spectrum).real
    quefrency = np.arange(len(cepstrum)) / sr
    
    if min_freq > 0:
        upper_bound = 1 / min_freq
    else:
        upper_bound = np.inf  # min_freq=0인 경우 상한을 무한대로 설정
    
    idx = (quefrency >= 1 / max_freq) & (quefrency <= upper_bound)
    
    fig, ax = plt.subplots(figsize=(8, 4))  
    ax.plot(quefrency[idx], cepstrum[idx])
    ax.set_title('Cepstrum')
    ax.set_xlabel('Quefrency (s)')
    ax.set_ylabel('Amplitude')
    ax.grid(True)
    ax.set_xlim(quefrency[idx].min(), quefrency[idx].max())
    ax.set_ylim(np.min(cepstrum[idx]), np.max(cepstrum[idx]))
    plt.tight_layout()
    return fig, ax

def compute_and_plot_mel_spectrogram(y: np.ndarray, sr: int, display_axes: bool):
    S = librosa.feature.melspectrogram(y=y, sr=sr, fmin=0, fmax=8000)
    S_db = librosa.power_to_db(S, ref=np.max)

    if not display_axes:
        figsize = (5.12, 5.12)  # 5.12 x 5.12 inches
        dpi = 100
    else:
        # 축이 있는 버전과 비율을 맞추기 위해 figsize 조정
        figsize = (6.4, 5.12)  # 예시로 너비를 조금 늘려서 비율을 맞춤
        dpi = 100

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    img = librosa.display.specshow(
        S_db,
        sr=sr,
        fmin=0,
        fmax=8000,
        ax=ax,
    )
    if not display_axes:
        ax.axis('off')
    else:
        ax.set_title('Mel Spectrogram')
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        # 축 범위 설정
    plt.tight_layout(pad=0)
    return fig, ax

def compute_and_plot_linear_spectrogram(y: np.ndarray, sr: int, n_fft: int, hop_length: int, display_axes: bool):
    S = np.abs(librosa.stft(
        y=y,
        n_fft=n_fft,
        hop_length=hop_length,
        window='hann',
    ))
    S_db = librosa.amplitude_to_db(S, ref=np.max)

    if not display_axes:
        figsize = (5.12, 5.12)  # 5.12 x 5.12 inches
        dpi = 100
    else:
        # 축이 있는 버전과 비율을 맞추기 위해 figsize 조정
        figsize = (6.4, 5.12)  # 예시로 너비를 조금 늘려서 비율을 맞춤
        dpi = 100

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    img = librosa.display.specshow(
        S_db,
        sr=sr,
        hop_length=hop_length,
        x_axis='time',
        y_axis='linear',
        ax=ax,
    )
    if not display_axes:
        ax.axis('off')
    else:
        ax.set_title('Linear Spectrogram')
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        # 축 범위 설정
        ax.set_xlim(0, len(y)/sr)
        ax.set_ylim(0, sr/2)
    plt.tight_layout(pad=0)
    return fig, ax

def compute_and_plot_mfcc(y: np.ndarray, sr: int, normalization: str = "min-max"):
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    if normalization == "min-max":
        mfccs_normalized = (mfccs - np.min(mfccs)) / (np.max(mfccs) - np.min(mfccs) + 1e-10)
    elif normalization == "z-score":
        mfccs_normalized = (mfccs - np.mean(mfccs)) / (np.std(mfccs) + 1e-10)
    else:
        raise ValueError("Unsupported normalization type. Use 'min-max' or 'z-score'.")
    fig, ax = plt.subplots(figsize=(8, 4))  
    img = librosa.display.specshow(
        mfccs_normalized, 
        x_axis='time', 
        ax=ax, 
        cmap='coolwarm'
    )
    ax.set_title('MFCC')
    fig.colorbar(img, ax=ax)
    ax.grid(True)
    ax.set_ylim(np.min(mfccs_normalized), np.max(mfccs_normalized))
    plt.tight_layout(pad=0)
    return fig, ax

def compute_and_plot_chroma_features(y: np.ndarray, sr: int):
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    fig, ax = plt.subplots(figsize=(8, 4))  
    img = librosa.display.specshow(
        chroma, 
        x_axis='time', 
        y_axis='chroma', 
        cmap='coolwarm', 
        ax=ax
    )
    ax.set_title('Chroma Features')
    fig.colorbar(img, ax=ax)
    ax.grid(True)
    ax.set_ylim(0, 12)  # 12개의 크로마 피처
    plt.tight_layout(pad=0)
    return fig, ax

def compute_and_plot_spectral_centroid(y: np.ndarray, sr: int):
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    times = librosa.times_like(spectral_centroids)
    fig, ax = plt.subplots(figsize=(8, 4))  
    ax.plot(times, spectral_centroids)
    ax.set_title('Spectral Centroid')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Frequency (Hz)')
    ax.grid(True)
    ax.set_ylim(np.min(spectral_centroids), np.max(spectral_centroids))
    plt.tight_layout()
    return fig, ax

def compute_and_plot_spectral_bandwidth(y: np.ndarray, sr: int):
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
    times = librosa.times_like(spectral_bandwidth)
    fig, ax = plt.subplots(figsize=(8, 4))  
    ax.plot(times, spectral_bandwidth)
    ax.set_title('Spectral Bandwidth')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Bandwidth (Hz)')
    ax.grid(True)
    ax.set_ylim(np.min(spectral_bandwidth), np.max(spectral_bandwidth))
    plt.tight_layout()
    return fig, ax

def compute_and_plot_power_spectrum(y: np.ndarray, sr: int):
    S = np.abs(np.fft.fft(y))**2
    freqs = np.fft.fftfreq(len(y), 1/sr)
    idx = np.argsort(freqs)
    idx = idx[freqs[idx] >= 0]
    freqs = freqs[idx]
    S = S[idx]
    fig, ax = plt.subplots(figsize=(8, 4))  
    ax.plot(freqs, S)
    ax.set_title('Power Spectrum')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power')
    ax.grid(True)
    ax.set_xlim(0, sr/2)
    ax.set_ylim(0, np.max(S))
    plt.tight_layout()
    return fig, ax

def create_json_structure(data_entries, output_json_path):
    """
    Create a JSON file with structured data entries.

    Parameters:
        data_entries (list): List of feature dictionaries.
        output_json_path (str): Path to save the JSON file.
    """
    def convert_numpy_types(obj):
        if isinstance(obj, np.float32) or isinstance(obj, np.float64):
            return float(obj)
        elif isinstance(obj, dict):
            return {k: convert_numpy_types(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(i) for i in obj]
        else:
            return obj

    # JSON 저장 전에 변환
    data = convert_numpy_types({"audio_features": data_entries})
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    main()


백업 완료: 'audio_features_backup.json'
사용 가능한 CPU 코어 수: 12, 사용되는 코어 수: 11
총 처리할 엔트리 수: 8400


Processing entries: 100%|██████████| 8400/8400 [51:07<00:00,  2.74it/s]  



JSON 파일이 'audio_features_re.json' 경로에 저장되었습니다.


# 단순히 librosa parameter 만 json에 추가하려면?

In [2]:
import json
import sys

def load_json(json_path):
    """
    Load JSON data from a file.

    Parameters:
        json_path (str): Path to the JSON file.

    Returns:
        dict: Loaded JSON data.
    """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"Error loading JSON: {e}", file=sys.stderr)
        sys.exit(1)

def save_json(data, json_path):
    """
    Save JSON data to a file.

    Parameters:
        data (dict): JSON data to save.
        json_path (str): Path to save the JSON file.
    """
    try:
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"JSON successfully saved to '{json_path}'.")
    except Exception as e:
        print(f"Error saving JSON: {e}", file=sys.stderr)
        sys.exit(1)

def transform_features(features):
    """
    Transform image feature paths into structured dictionaries.

    Parameters:
        features (dict): Original features dictionary.

    Returns:
        dict: Transformed features dictionary.
    """
    # List of image feature keys
    image_features = [
        "waveform",
        "envelope",
        "cepstrum",
        "mel_spectrogram_with_axes",
        "mel_spectrogram_no_axes",
        "mfcc",
        "linear_spectrogram_with_axes",
        "linear_spectrogram_no_axes",
        "chroma_features",
        "spectral_centroid",
        "spectral_bandwidth",
        "power_spectrum"
    ]
    
    # Define librosa_parameters and plot_parameters for each image feature
    feature_parameters = {
        "waveform": {
            "librosa_parameters": {},
            "plot_parameters": {
                "figsize": [8, 4],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "envelope": {
            "librosa_parameters": {},
            "plot_parameters": {
                "figsize": [8, 4],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "cepstrum": {
            "librosa_parameters": {
                "min_freq": 0,
                "max_freq": 8000
            },
            "plot_parameters": {
                "figsize": [8, 4],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "mel_spectrogram_with_axes": {
            "librosa_parameters": {
                "fmin": 0,
                "fmax": 8000
            },
            "plot_parameters": {
                "figsize": [6.4, 5.12],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "mel_spectrogram_no_axes": {
            "librosa_parameters": {
                "fmin": 0,
                "fmax": 8000
            },
            "plot_parameters": {
                "figsize": [5.12, 5.12],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "mfcc": {
            "librosa_parameters": {
                "n_mfcc": 20,
                "normalization": "min-max"
            },
            "plot_parameters": {
                "figsize": [8, 4],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "linear_spectrogram_with_axes": {
            "librosa_parameters": {
                "n_fft": 256,
                "hop_length": 256,
                "window": "hann"
            },
            "plot_parameters": {
                "figsize": [6.4, 5.12],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "linear_spectrogram_no_axes": {
            "librosa_parameters": {
                "n_fft": 256,
                "hop_length": 256,
                "window": "hann"
            },
            "plot_parameters": {
                "figsize": [5.12, 5.12],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "chroma_features": {
            "librosa_parameters": {
                "hop_length": 512
            },
            "plot_parameters": {
                "figsize": [8, 4],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "spectral_centroid": {
            "librosa_parameters": {
                "hop_length": 512
            },
            "plot_parameters": {
                "figsize": [8, 4],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "spectral_bandwidth": {
            "librosa_parameters": {
                "hop_length": 512
            },
            "plot_parameters": {
                "figsize": [8, 4],
                "dpi": 100,
                "file_format": "png"
            }
        },
        "power_spectrum": {
            "librosa_parameters": {
                "n_fft": "len(y)",
                "hop_length": None
            },
            "plot_parameters": {
                "figsize": [8, 4],
                "dpi": 100,
                "file_format": "png"
            }
        }
    }

    transformed = {}
    for key, value in features.items():
        if key in image_features:
            transformed[key] = {
                "path": value,
                "librosa_parameters": feature_parameters[key]["librosa_parameters"],
                "plot_parameters": feature_parameters[key]["plot_parameters"]
            }
        else:
            transformed[key] = value
    return transformed

def main():
    """
    Main function to transform the JSON structure.
    """
    input_json_path = 'audio_features_backup.json'  # Path to your existing JSON
    output_json_path = 'audio_features_transformed.json'  # Path for the transformed JSON

    # Load the existing JSON data
    data = load_json(input_json_path)
    audio_features = data.get("audio_features", [])

    if not audio_features:
        print("No 'audio_features' found in the JSON.", file=sys.stderr)
        sys.exit(1)

    updated_entries = []
    for entry in audio_features:
        features = entry.get("features", {})
        transformed_features = transform_features(features)
        entry['features'] = transformed_features
        updated_entries.append(entry)

    # Create the updated data structure
    updated_data = {"audio_features": updated_entries}

    # Save the transformed JSON
    save_json(updated_data, output_json_path)

if __name__ == "__main__":
    main()


JSON successfully saved to 'audio_features_transformed.json'.
